In [1]:
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
import torch
from torch.autograd import Variable
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import random
from PIL import Image
import copy
import pickle
import sys

from pycocotools.coco import COCO
import skimage.io
import io
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

from tqdm import tqdm

In [2]:
dataDir='../coco_dataset/'
imagesDirTrain = '{}train2017'.format(dataDir)
imagesDirVal = '{}val2017'.format(dataDir)

annTrainFile = '{}annotations/captions_train2017.json'.format(dataDir)
annValFile = '{}annotations/captions_val2017.json'.format(dataDir)

transform = transforms.Compose([transforms.Resize((224, 224)), 
                                             transforms.ToTensor(), 
                                             transforms.Normalize(
                                                 mean=[0.485, 0.456, 0.406],
                                                 std=[0.229, 0.224, 0.225])])

random.seed(1234)

In [2]:
def numpy2image(img_numpy):
    if img_numpy.dtype == np.dtype('float64'):
        img_numpy = (img_numpy*255).astype('uint8')
    return Image.fromarray(img_numpy)


class MSCOCODataset(Dataset):
    """MSCOCO Dataset"""

    def __init__(self, annFile, imagesDir, transform=None):
        self.coco = COCO(annFile)
        self.imagesDir = imagesDir
        self.imageids = self.coco.getImgIds()
        self.transform = transform

    def __len__(self):
        return len(self.coco.dataset['images'])

    def __getitem__(self, idx):
        imid = self.imageids[idx]
        img_data = self.coco.loadImgs([imid])[0]
        
        img_file_name = '{}/{}'.format(self.imagesDir, img_data['file_name'])
        image = skimage.io.imread(img_file_name)
        
        if len(image.shape) != 3:
            return self.__getitem__(0)
        
        image = numpy2image(image)
        if self.transform is not None:
            image = self.transform(image)
            
        sample = {'image': image, 'id': imid}
            
        return sample

In [4]:
train_dataset = MSCOCODataset(annTrainFile, imagesDirTrain, transform)
test_dataset = MSCOCODataset(annValFile, imagesDirVal, transform)

train_dataloader = DataLoader(train_dataset, batch_size=32)
test_dataloader = DataLoader(test_dataset, batch_size=32)

loading annotations into memory...
Done (t=2.76s)
creating index...
index created!
loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


In [5]:
def images2vec(loader, cnn):        
    res = None
    res_ids = None
    
    for item in tqdm(loader):
            
        X = Variable(item['image'].type(torch.cuda.FloatTensor))
        ids = item['id']
            
        vec = cnn.forward(X).data.cpu()
        vec = vec.view(vec.size(0), -1)
        
        if res is None:
            res = vec
            res_ids = ids
        else:s
            res = torch.cat((res, vec), 0)
            res_ids = torch.cat((res_ids, ids), 0)
    return res, res_ids

In [6]:
cnn = models.resnet152(pretrained=True)
cnn = nn.Sequential(*list(cnn.children())[:-1])

In [7]:
train, train_ids = images2vec(train_dataloader, cnn)
test, test_ids = images2vec(test_dataloader, cnn)

100%|██████████| 157/157 [01:54<00:00,  1.38it/s]


In [8]:
torch.save(train, 'train_image_resnet.pth')

In [9]:
torch.save(train_ids, 'train_image_ids_resnet.pth')

In [10]:
torch.save(test, 'test_image_resnet.pth')

In [11]:
torch.save(test_ids, 'test_image_ids_resnet.pth')